#Image Classification Task:
• Load the MNIST dataset.
• Build a simple convolutional neural network (CNN) using Keras Sequential model.
• Train the CNN model on the MNIST dataset.
• Evaluate the model's performance on a test set and report accuracy.
• Use grid search to optimize hyperparameters such as learning rate, batch size, and
optimizer choice.
• Use Callback functions to automate training process like “ReduceLROnPlateau” and keep
check on validation loss. Also use history object for result visualization.


In [1]:
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape to [samples][width][height][channels]
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

# Normalize from 0-255 to 0-1
X_train /= 255
X_test /= 255

# One hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]


ModuleNotFoundError: No module named 'keras'

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from keras.callbacks import ReduceLROnPlateau

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Train
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, callbacks=[reduce_lr], verbose=2)


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Load and preprocess data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

def build_model(learning_rate=0.001):
    model = Sequential([
        Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Grid search settings
learning_rates = [0.001, 0.0001]
batch_sizes = [32, 64]

# Grid search
best_accuracy = 0
best_lr = 0
best_batch = 0

for lr in learning_rates:
    for batch_size in batch_sizes:
        model = build_model(learning_rate=lr)
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=batch_size, verbose=0)
        accuracy = np.max(history.history['val_accuracy'])
        print(f"LR={lr}, Batch={batch_size}, Val Accuracy={accuracy}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_lr = lr
            best_batch = batch_size

print(f"Best parameters: LR={best_lr}, Batch={best_batch}, with accuracy={best_accuracy}")


In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


#Text Classification Task:
* Load the IMDb movie reviews dataset.
* Preprocess the text data (tokenization, padding).
* Build a simple recurrent neural network (RNN) for text classification.
* Train the RNN model on the IMDb dataset.
* Evaluate the model's performance on a test set and report accuracy.
* Use grid search to optimize hyperparameters such as learning rate, batch size, and
optimizer choice.
* Use Callback functions to automate training process like “ReduceLROnPlateau” and keep
check on validation loss. Also use history object for result visualization.

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset
vocab_size = 10000  # Consider only the top 10,000 words in the dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Inspect
print(f"Training samples: {len(x_train)}")
print(f"Test samples: {len(x_test)}")


In [ ]:
max_length = 500  # Maximum length of all sequences

x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

def build_rnn():
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=32, input_length=max_length),
        SimpleRNN(32),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_rnn()
model.summary()


In [ ]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam, SGD
import numpy as np

def build_rnn(optimizer='adam', learning_rate=0.001):
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Unsupported optimizer")

    model = Sequential([
        Embedding(input_dim=10000, output_dim=32, input_length=500),
        SimpleRNN(32),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

learning_rates = [0.001, 0.0001]
batch_sizes = [64, 128]
optimizers = ['adam', 'sgd']

best_acc = 0
best_params = {}

for optimizer in optimizers:
    for lr in learning_rates:
        for batch_size in batch_sizes:
            print(f"Training with optimizer={optimizer}, learning_rate={lr}, batch_size={batch_size}")
            model = build_rnn(optimizer=optimizer, learning_rate=lr)
            model.fit(x_train, y_train, epochs=3, batch_size=batch_size, validation_split=0.2, verbose=0)
            loss, acc = model.evaluate(x_test, y_test, verbose=0)
            print(f"Accuracy: {acc}, Loss: {loss}")

            if acc > best_acc:
                best_acc = acc
                best_params = {'optimizer': optimizer, 'learning_rate': lr, 'batch_size': batch_size}

print(f"Best accuracy: {best_acc} with parameters {best_params}")


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=[reduce_lr])


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()
